# Broken

In [12]:
import os
import signal
import time

class TaskQueue:
  def __init__(self, workers_cnt = os.cpu_count()):
    self.worker_cnt = workers_cnt
    self.pids = []
    self.tasks = []
  
  def enqueue(self, func, args):
    self.tasks += [(func, args)]
    self._clear_completed()
    self._run_queued()
  
  def join(self):
    while self.tasks and self.pids:
      self._clear_completed()
      self._run_queued()
      status = os.wait()
      if status[0] in self.pids:
        self.pids.remove(status[0])
      else:
        print(f'Joined a spurious zombie {status[0]}')

  def _run_queued(self):
    self.pids += [self._run(*self.tasks.pop(0)) for i in range(self.worker_cnt - len(self.pids)) if self.tasks]
  
  def _clear_completed(self):
    self.pids = [pid for pid in self.pids if os.waitpid(pid, os.WNOHANG)[0] == 0]

  def _run(self, func, args):
    pid = os.fork()
    if pid == 0:
      func(*args)
      self._exit()
    return pid

  def _exit(self):
    while True:
      os.kill(0, signal.SIGTERM)
      time.sleep(1)
      os.kill(0, signal.SIGKILL)

In [13]:
import time

def proc(x):
    time.sleep(1)

tq = TaskQueue()
for i in range(100):
  tq.enqueue(proc, i)

t1 = time.perf_counter()
tq.join()
t2 = time.perf_counter()
print(t2-t1)

Joined a spurious zombie 357
Joined a spurious zombie 358
Joined a spurious zombie 359
Joined a spurious zombie 360


KeyboardInterrupt: 